# Implementing some abstractions
At this stage, I claim we've stumbled on a pattern that we might want to re-use. Vega / lite publish a [comprehensive](https://vega.github.io/vega/examples/) set of [examples](https://vega.github.io/vega-lite/examples/) - we want to start from them, modify them, and plot our chart in some "context". 

One idea would be to [write a script](https://github.com/Quafadas/dedav4s/blob/main/core/jvm/src/main/scala/viz/genVegaPlots.sc) which scrapes the ~~examples~~ [links to the examples](https://github.com/Quafadas/dedav4s/blob/main/core/shared/src/main/scala/viz/vega/plots/SpecUrls.scala) off the vega/lite website. Given that, you need only a very simple http library, to go fetch those battle-tested-part-of-the-core-vega-library starting points.

We'll pragmatically accept mutability and [iterate around](https://github.com/Quafadas/dedav4s/blob/861c3fa38f41084f9d2e1ea168da40aab22eccf5/core/shared/src/main/scala/viz/WithBaseSpec.scala#L31) a ```Seq[ujson.Value => Unit]``` (a list of side effects) as a way to apply changes to the spec. 

Context sounds a lot like `implicit` / `given` to my mind. Let's call it a ["plot target"](https://github.com/Quafadas/dedav4s/blob/861c3fa38f41084f9d2e1ea168da40aab22eccf5/core/jvm/src/main/scala/viz/PlotTarget.scala#L122)...

And with all this machinery we could write something which both abstracts away the messy business of the plot itself, and keeps the intent pretty clear. Let's plot the distrubition of html tags for some arbitrary website. 

In [18]:
import $ivy.`org.jsoup:jsoup:1.15.3`
import $ivy.`io.github.quafadas::dedav4s:0.9-2176b4b-20230421T114100Z-SNAPSHOT`

import org.jsoup._
import collection.JavaConverters._
import viz.PlotTargets.almond
import viz.vega.plots.PieChart

def plotElementDist(url:String) =
  val doc = Jsoup.connect(url).get
  val els = doc.body().children().select("*").asScala
  val counted = els.groupMapReduce(_.tag.toString())(_ => 1)(_ + _)
  val data = for ((label, number) <- counted)
    yield ujson.Obj(
      "id" -> label,
      "field" -> number.toDouble
    )
  // This "PieChart" case class, is the abstraction described above.  
  PieChart(
    List(      
      spec => spec("height") = 600,
      spec => spec("width") = 600,
      (spec: ujson.Value) => spec("data")(0)("values") = data
    )
  )

import $ivy.$                       

import $ivy.$                                  


import org.jsoup._

import collection.JavaConverters._

import viz.PlotTargets.almond

import viz.vega.plots.PieChart


defined function plotElementDist

In [21]:
plotElementDist("https://google.com")
//val plot = plotElementDist("https://google.com")
// show(plot.spec) // for debugging a troublesome spec

res20: PieChart = PieChart(
  mods = List(
    ammonite.$sess.cmd17$Helper$$Lambda$3590/0x00000008016f2e68@7ec1f41b,
    ammonite.$sess.cmd17$Helper$$Lambda$3591/0x00000008016f3268@136d214f,
    ammonite.$sess.cmd17$Helper$$Lambda$3592/0x00000008016f3668@31d934b8
  )
)

In [20]:
plotElementDist("https://www.scala-lang.org")

res19: PieChart = PieChart(
  mods = List(
    ammonite.$sess.cmd17$Helper$$Lambda$3590/0x00000008016f2e68@7ec1f41b,
    ammonite.$sess.cmd17$Helper$$Lambda$3591/0x00000008016f3268@136d214f,
    ammonite.$sess.cmd17$Helper$$Lambda$3592/0x00000008016f3668@335ba5df
  )
)

I quite like this, if you want to keep the data acquisition, and plotting as seperate concerns. The "intent" here is, to my eye, rather readable. Whilst plotting the html element counts of popular websites is... pointless, I found it extended nicely to other, more helpful problems :-).

## Elephant

There is however, a rather large elephant in the room - for this to work "for you", you have to be willing to do the legwork of learning vega. Whilst easy enough to trace the spec back, and find the "example" this refers to... I can't see a shortcut here for needing to understand and ultimately innovate on vega. That is actually a pretty fun undertaking... but it's (at least) a multiple day time investment. (A worthwhile one, I claim!)

It isn't however realllllly ... "plotting in scala"... more like using scala to write vega.